# Day 21
https://adventofcode.com/2015/day/21

In [1]:
import aocd
data = aocd.get_data(year=2015, day=21)

In [2]:
from dataclasses import dataclass
from itertools import chain, combinations, product
from typing import Set

##### Part 1: Win the game as the Player

In [59]:
@dataclass
class Boss():
    hitpoints: int
    damage: int
    armor: int
        
    def __init__(self, desc):
        for line in desc.split('\n'):
            attribute, value = line.split(': ')
            if attribute == 'Hit Points':
                self.hitpoints = int(value)
            elif attribute == 'Damage':
                self.damage = int(value)
            elif attribute == 'Armor':
                self.armor = int(value)
    
    def hit(self, damage: int) -> bool:
        self.hitpoints -= max(1, damage - self.armor)
        return self.hitpoints <= 0

In [19]:
def boss_factory(data):
    def factory():
        return Boss(data)
    return factory

In [4]:
@dataclass
class Equipment():
    name: int
    cost: int
    damage: int
    armor: int
        
    def __hash__(self):
        return hash(self.name)

weapons = {
    Equipment(name='Dagger', cost=8, damage=4, armor=0),
    Equipment(name='Shortsword', cost=10, damage=5, armor=0),
    Equipment(name='Warhammer', cost=25, damage=6, armor=0),
    Equipment(name='Longsword', cost=40, damage=7, armor=0),
    Equipment(name='Greataxe', cost=74, damage=8, armor=0),
}
armor = {
    Equipment(name='Leather', cost=13, damage=0, armor=1),
    Equipment(name='Chainmail', cost=31, damage=0, armor=2),
    Equipment(name='Splintmail', cost=53, damage=0, armor=3),
    Equipment(name='Bandedmail', cost=75, damage=0, armor=4),
    Equipment(name='Platemail', cost=102, damage=0, armor=5),
}
rings = {
    Equipment(name='Damage +1', cost=25, damage=1, armor=0),
    Equipment(name='Damage +2', cost=50, damage=2, armor=0),
    Equipment(name='Damage +3', cost=100, damage=3, armor=0),
    Equipment(name='Defense +1', cost=20, damage=0, armor=1),
    Equipment(name='Defense +2', cost=40, damage=0, armor=2),
    Equipment(name='Defense +3', cost=80, damage=0, armor=3),
}

In [5]:
@dataclass
class Player():
    inventory: Set[Equipment]
    hitpoints: int
    
    @property
    def armor(self) -> int:
        return sum(item.armor for item in self.inventory)
    
    @property
    def damage(self) -> int:
        return sum(item.damage for item in self.inventory)
    
    @property
    def spent(self) -> int:
        return sum(item.cost for item in self.inventory)
    
    def hit(self, damage: int) -> bool:
        self.hitpoints -= max(1, damage - self.armor)
        return self.hitpoints <= 0

In [6]:
def fight_won(boss: Boss, player: Player) -> bool:
    while True:
        if boss.hit(player.damage):
            return True
        if player.hit(boss.damage):
            return False

In [12]:
def all_loadouts():
    weapon_options = tuple(combinations(weapons, 1))
    armor_options = tuple(chain([tuple()], combinations(armor, 1)))
    ring_options = tuple(chain([tuple()], combinations(rings, 1), combinations(rings, 2)))
    return tuple(Player(hitpoints=100, inventory=set(w + a + r))
                 for w, a, r in product(weapon_options, armor_options, ring_options))

In [69]:
def best_win(boss_factory):
    for player in sorted(all_loadouts(), key=lambda p: p.spent):
        if fight_won(boss_factory(), player):
            return player

In [71]:
bf = boss_factory(data)
p1 = best_win(bf)
print('Part 1: {}'.format(p1.spent))

Part 1: 78


##### Part 2: Win the game as the shopkeeper

In [79]:
def worst_loss(boss_factory):
    for player in sorted(all_loadouts(), key=lambda p: p.spent, reverse=True):
        if not fight_won(boss_factory(), player):
            return player

In [80]:
p2 = worst_loss(bf)
print('Part 2: {}'.format(p2.spent))

Part 2: 148
